In [2]:
################################################################################################
import csv
import cv2
import os
import requests
import numpy as np 
import imutils 
import cv2
import torch
# from upload_attendence import *
from facenet_pytorch import MTCNN
url = "http://192.168.100.138:8080/shot.jpg"
import pickle
import time
from threading import Thread
import threading
import os
import cvzone
from datetime import datetime,timedelta
import numpy as np
import face_recognition
import upload_attendence

# def task1():
# cap=cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FPS, 144)
# cap.set(3,140)
# cap.set(4,20)
imgB_size=cv2.imread("layout.png")
height, width, channels = imgB_size.shape

width, height = 600, 395
width1, height1 =460, 395

# print(imgB_size.shape)
imgB_ = cv2.resize(imgB_size, (1565,795))

# folderModePath="resources/modes/"
folderModePath="faces_detected/"
modePathList=os.listdir(folderModePath)
imgModeList=[]
paths1=[]
for path in modePathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath,path)))
    paths1.append(path)



paths2=[]
folderImagePath="faces_detected/"
modeImageList=os.listdir(folderImagePath)
imageList=[]
for path in modeImageList:
    imageList.append(cv2.imread(os.path.join(folderImagePath,path)))
    paths2.append(path)
print(paths2)

paths3=[]
folderImagePath="face_detected3/"
modeImageList1=os.listdir(folderImagePath)
for path in modeImageList1:
    paths3.append(path)
print(len(paths3))
superb=paths3.copy()



employee=paths2.copy()
print(employee)
count=0
now=datetime.now()

current_date=now.strftime("%Y-%m-%d")
print(current_date)

# with open(current_date+'.csv', 'w+', newline='') as f:
#     writer = csv.writer(csvfile)

f=open(current_date+'.csv','a',newline='')
writer=csv.writer(f)

# load encoding file
file=open("EncodeFile.p","rb")
encodeListknownId=pickle.load(file)
file.close()
encodeListknown,studentId=encodeListknownId
# print(studentId)
print("Encode File Load!")




file=open("EncodeFile2.p","rb")
encodeListKnownId=pickle.load(file)
file.close()
encodeListKnown,StudentId=encodeListKnownId
# print(studentId)
print("Encode File Load!")

cap=cv2.VideoCapture(0)
while True:
    success,img=cap.read()
    
#     img_resp = requests.get(url) 
#     img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8) 
#     img = cv2.imdecode(img_arr, -1) 
#     img = imutils.resize(img, width=1000, height=1800) 
    
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to remove noise
    blur_img = cv2.GaussianBlur(gray_img, (5, 5), 0)

    # Apply histogram equalization to improve contrast
    eq_img = cv2.equalizeHist(blur_img)

    # Apply non-local means filtering for denoising
    nlm_img = cv2.fastNlMeansDenoising(eq_img, None, 10, 7, 21)

    # Apply gamma correction for image enhancement
    gamma = 0.5
    gamma_img = cv2.pow(nlm_img/255.0, gamma)
    gamma_img = (gamma_img*255).astype('uint8')
    
    scale_factor = 2.0
    
    # Resize the image
    resized_img = cv2.resize(gamma_img, (0,0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
    rgb_img = cv2.cvtColor(resized_img, cv2.COLOR_GRAY2RGB)
    
    
    
    
    imgs=cv2.resize(rgb_img,(0,0),None,0.25,0.25)
    imgs=cv2.cvtColor(imgs,cv2.COLOR_BGR2RGB) #face detection takes RGB

    faceCurFrame=face_recognition.face_locations(imgs)
    encodeCurFrame=face_recognition.face_encodings(imgs,faceCurFrame)

    img1= cv2.resize(rgb_img, (width, height))
    imgB_[260:height+260, 50:50+width] = img1

    if len(faceCurFrame)!=0:

        
        for encodeFace,faceLoc in zip(encodeCurFrame,faceCurFrame): #simultaneously run for loop
            matches=face_recognition.compare_faces(encodeListknown,encodeFace)
            faceDis=face_recognition.face_distance(encodeListknown,encodeFace)
            print("matches:",matches)
    #         print("FaceDist:",faceDis)

            matchIndex=np.argmin(faceDis)
            print("MatchIndex:",matchIndex)
            
            
            image=cv2.imread(("resources/attendence_mark.png"))
            c=cv2.imread("resources/modes/dash.png")
            m=cv2.imread("resources/modes/active.png")
            if True in matches:
                m=None

                img_New="faces_detected/"+paths2[matchIndex]
                if paths2[matchIndex] in employee:
                    time.sleep(1)
                    img_new=cv2.imread(str(img_New))
                    widthw,heighth,_=img_new.shape

#                     img_= cv2.resize(img_new, (widthw, heighth))

#                     widthh,heighthh,_=img_.shape

                    imgB_[180:180+(widthw), 1000:1000+(heighth)] = img_new

                    employee.remove(paths2[matchIndex])
                    print(employee)
                    y1,x2,y2,x1=faceLoc

                    y1,x2,y2,x1= y1*4,x2*4,y2*4,x1*4

                    bbox = 50+x1,172+y1,x2-x1,y2-y1

                    imgB_ =cvzone.cornerRect(imgB_,bbox,rt=0)

                    widthh,heighthh,_=image.shape


                    imgB_[260:260+(widthh), 1200:1200+(heighthh)] = image

                    w,h,_=c.shape
                    new=cv2.resize(c,(widthh,heighthh))
                    
                    current_time=now.strftime("%H-%M-%S")
                    print(current_time)
                    
                    with open('attendence.csv', 'a') as f:
                        writer=csv.writer(f)
                        writer.writerow([paths2[matchIndex],current_date,current_time])
                    
                else:
                
                    y1,x2,y2,x1=faceLoc

                    y1,x2,y2,x1= y1*4,x2*4,y2*4,x1*4

                    bbox = 50+x1,172+y1,x2-x1,y2-y1

                    imgB_ =cvzone.cornerRect(imgB_,bbox,rt=0)

                    n=image
                    w,h,_=n.shape
                    new=cv2.resize(image,(w,h))
                    imgB=cv2.imread("layout.png")
                    imgB_ = cv2.resize(imgB_size, (1565,795))
                    img=cv2.resize(img,(width, height))
                    imgB_[260:height+260, 50:50+width] = img

                    imgB_[260:260+(w), 1200:1200+(h)] =new

            else:
                
                for encodeFace,faceLoc in zip(encodeCurFrame,faceCurFrame): #simultaneously run for loop
                    matches1=face_recognition.compare_faces(encodeListKnown,encodeFace)
                    faceDis=face_recognition.face_distance(encodeListKnown,encodeFace)
                    print("matches:",matches1)
            #         print("FaceDist:",faceDis)
                    
#                     matchIndex=np.argmin(faceDis)
#                     print("MatchIndex:",matchIndex)


                    image=cv2.imread(("resources/attendence_mark.png"))
                    c=cv2.imread("resources/modes/dash.png")
                    m=cv2.imread("resources/modes/active.png")
                    if True in matches1:
                        matchIndex=np.argmin(faceDis)
                        print("MatchIndex:",matchIndex)
                        m=None
                        img_New="face_detected3/"+paths3[matchIndex]

                        img_new=cv2.imread(str(img_New))
                        widthw,heighth,_=img_new.shape

                        imgB_[180:180+(widthw), 1000:1000+(heighth)] = img_new

#                             superb.remove(paths3[matchIndex])
#                             print(superb)
                        y1,x2,y2,x1=faceLoc

                        y1,x2,y2,x1= y1*4,x2*4,y2*4,x1*4

                        bbox = 50+x1,172+y1,x2-x1,y2-y1

                        imgB_ =cvzone.cornerRect(imgB_,bbox,rt=0)

                        widthh,heighthh,_=image.shape


                        imgB_[260:260+(widthh), 1200:1200+(heighthh)] = image

                        w,h,_=c.shape
                        new=cv2.resize(c,(widthh,heighthh))

                        current_time=now.strftime("%H-%M-%S")
                        print(current_time)

                        with open('attendence2.csv', 'a') as f:
                            writer=csv.writer(f)
                            writer.writerow([paths2[matchIndex],current_date,current_time])  
                    else:




                        folder_path = "face_detected2/"
                        file_count = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])



                        filename = f'face_detected2/face_{file_count}.jpg'
                        file_count+=1
                        cv2.imwrite(filename, rgb_img)
                        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
                        mtcnn = MTCNN(keep_all=True, device=device)

                        # Set minimum and maximum face sizes and confidence threshold
                        min_face_size = 0
                        max_face_size = 1000
                        confidence_threshold = [0.9, 0.95, 0.95]
                        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                        # Detect faces in the RGB frame using the MTCNN detector
                        boxes, _ = mtcnn.detect(rgb, landmarks=False)


                        # Get the average face size in the current frame
                        if boxes is not None:
                            face_sizes = [box[2] - box[0] for box in boxes]
                            if len(face_sizes) > 0:
                                avg_face_size = sum(face_sizes) / len(face_sizes)
                                min_face_size = max(30, int(avg_face_size * 0.7))
                                max_face_size = int(avg_face_size * 1.3)

                        folder_path = "face_detected3/"
                        file_count1 = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

                        cunts=file_count1
                        # Draw rectangles around the detected faces
                        if boxes is not None:

                            for box in boxes:
                                x1, y1, x2, y2 = box.astype('int')
                                face_size = x2 - x1
                                if min_face_size <= face_size <= max_face_size:
                                    cv2.rectangle(img, (x1-50, y1-50), (x2+50, y2+50), (0, 255, 0), 2)

                                    face = img[y1:y2, x1:x2]

                                    filename = f'face_detected3/face_{cunts}.jpg'
                                    cv2.imwrite(filename, face)
                                    cunts+=1






                    count+=1
                    n=image
                    image=None
                    c=None

                    m=cv2.imread("resources/modes/active.png")

                    w,h,_=n.shape
                    new=cv2.resize(m,(w,h))
                    imgB=cv2.imread("layout.png")
                    imgB_ = cv2.resize(imgB_size, (1565,795))
                    img=cv2.resize(img,(width, height))
                    imgB_[260:height+260, 50:50+width] = img

                    imgB_[260:260+(w), 1200:1200+(h)] =new
    else:
        imgB=cv2.imread("layout.png")
        imgB_ = cv2.resize(imgB_size, (1565,795))
        img=cv2.resize(img,(width, height))
        imgB_[260:height+260, 50:50+width] = img

    cv2.imshow("Img",imgB_)
    if cv2.waitKey(1)==27:
        img=None
        break
# cap.release()
cv2.destroyAllWindows()
# image.upload()
######################################################################################################